In [7]:
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import fsolve
from ipywidgets import interact, FloatSlider, VBox, Button
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html

# Define functions for the Lagrangian multiplier problem
def objective(ai, psi, alpha, beta):
    return (ai**alpha) * (psi**beta)

def constraint(ai, psi, E, Pai, Ppsi):
    return E - Pai * ai - Ppsi * psi

def lagrangian(ai, psi, lbda, alpha, beta, E, Pai, Ppsi):
    return objective(ai, psi, alpha, beta) + lbda * constraint(ai, psi, E, Pai, Ppsi)

def equations(vars, alpha, beta, E, Pai, Ppsi):
    ai, psi, lbda = vars
    dL_dai = alpha * (ai**(alpha - 1)) * (psi**beta) - lbda * Pai
    dL_dpsi = beta * (ai**alpha) * (psi**(beta - 1)) - lbda * Ppsi
    dL_dlbda = constraint(ai, psi, E, Pai, Ppsi)
    return [dL_dai, dL_dpsi, dL_dlbda]

# Define the plotting function
def update_plot(alpha, E, Pai, Ppsi, AGI):
    beta = 1 - alpha
    u = 2  # Given constant
    initial_guess = [1, 1, 1]
    solution = fsolve(equations, initial_guess, args=(alpha, beta, E, Pai, Ppsi))
    ai_sol, psi_sol, lbda_sol = solution

    scale = max(E/Pai, E/Ppsi)
    
    psi_vals = np.linspace(0, scale, 500)
    ai_vals = (E - Ppsi * psi_vals) / Pai
    
    # ai function points
    psi_func_vals = np.linspace(0.1, scale, 500)  # Avoid division by zero
    ai_func_vals = (objective(ai_sol, psi_sol, alpha, beta)**(1/alpha)) / ((psi_func_vals**beta)**(1/alpha))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[psi_sol], y=[ai_sol], mode='markers', marker=dict(size=10, color='blue'), showlegend=False))
    fig.add_trace(go.Scatter(x=psi_vals, y=ai_vals, mode='lines', line=dict(color='red', dash='solid'), name='Energy Budget'))
    fig.add_trace(go.Scatter(x=psi_func_vals, y=ai_func_vals, mode='lines', line=dict(color='blue'), name='Constant Usefulness'))

    fig.add_trace(go.Scatter(x=[psi_sol, psi_sol], y=[0, ai_sol], mode='lines', line=dict(color='black', dash='dash'), showlegend=False))
    fig.add_trace(go.Scatter(x=[0, psi_sol], y=[ai_sol, ai_sol], mode='lines', line=dict(color='black', dash='dash'), showlegend=False))

    fig.add_trace(go.Scatter(x=[0, scale], y=[AGI, AGI], mode='lines', line=dict(color='green', dash='solid'), name='AGI'))

    fig.update_layout(title=f'Framework for Optimal AI Deployment on a Robot',
                      xaxis_title='all other battery-using features',
                      yaxis_title='AI',
                      xaxis=dict(range=[0, scale]),
                      yaxis=dict(range=[0, scale]),
                      width=1000,
                      height=800)
    clear_output()
    display(fig);

# Create the interactive sliders
alpha_slider = FloatSlider(min=0.2, max=0.8, step=0.1, value=0.5, description='AI Usefulness')
E_slider = FloatSlider(min=1, max=30, step=1, value=10, description='Energy Budget')
Pai_slider = FloatSlider(min=0.125, max=10, step=0.125, value=2, description='AI Energy Cost')
Ppsi_slider = FloatSlider(min=0.125, max=10, step=0.125, value=2, description='Other Energy Cost')
AGI_slider = FloatSlider(min=1, max=10, step=0.25, value=4, description='AGI')


# Update plot when sliders change
interact(update_plot,
         alpha=alpha_slider,
         E=E_slider,
         Pai=Pai_slider,
         Ppsi=Ppsi_slider,
         AGI=AGI_slider)


interactive(children=(FloatSlider(value=0.5, description='AI Usefulness', max=0.8, min=0.2), FloatSlider(value…

<function __main__.update_plot(alpha, E, Pai, Ppsi, AGI)>